In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RET']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('rearranged during transfection', 151),
 ('reverse electron transport', 44),
 ('resistance exercise training', 42),
 ('resonance energy transfer', 28),
 ('reverse electron transfer', 27),
 ('receptor tyrosine kinase', 12),
 ('reticulocyte', 8),
 ('retroperitoneal', 7),
 ('receptor', 7),
 ('rapid ethanol tolerance', 5),
 ('rearranged in transfection', 5),
 ('rearranged during transfection proto oncogene', 4),
 ('ret', 3),
 ('retardation', 3),
 ('retene', 3),
 ('radiationless energy transfer', 3),
 ('rearranged during transfection gene', 3)]

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-29 02:21:52] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'radiationless energy transfer': 'ungrounded',
  'rapid ethanol tolerance': 'ungrounded',
  'rearranged during transfection': 'HGNC:9967',
  'rearranged during transfection gene': 'HGNC:9967',
  'rearranged during transfection proto oncogene': 'HGNC:9967',
  'rearranged in transfection': 'HGNC:9967',
  'receptor': 'ungrounded',
  'receptor tyrosine kinase': 'HGNC:9967',
  'resistance exercise training': 'ungrounded',
  'resonance energy transfer': 'ungrounded',
  'ret': 'ungrounded',
  'retardation': 'ungrounded',
  'retene': 'ungrounded',
  'reticulocyte': 'ungrounded',
  'retroperitoneal': 'ungrounded',
  'reverse electron transfer': 'ungrounded',
  'reverse electron transport': 'ungrounded'},
 {'HGNC:9967': 'RET'},
 ['HGNC:9967']]

In [11]:
grounding_map, names, pos_labels = [{'radiationless energy transfer': 'ungrounded',
  'rapid ethanol tolerance': 'ungrounded',
  'rearranged during transfection': 'HGNC:9967',
  'rearranged during transfection gene': 'HGNC:9967',
  'rearranged during transfection proto oncogene': 'HGNC:9967',
  'rearranged in transfection': 'HGNC:9967',
  'receptor': 'ungrounded',
  'receptor tyrosine kinase': 'HGNC:9967',
  'resistance exercise training': 'ungrounded',
  'resonance energy transfer': 'ungrounded',
  'ret': 'ungrounded',
  'retardation': 'ungrounded',
  'retene': 'ungrounded',
  'reticulocyte': 'ungrounded',
  'retroperitoneal': 'ungrounded',
  'reverse electron transfer': 'ungrounded',
  'reverse electron transport': 'ungrounded'},
 {'HGNC:9967': 'RET'},
 ['HGNC:9967']]

In [12]:
excluded_longforms = ['ret']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[]

In [20]:
intersection2

[]

In [1]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

NameError: name 'unambiguous_agent_texts' is not defined

In [18]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [21]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 02:24:27] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 02:24:31] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9668702162598457 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [22]:
classifier.stats

{'label_distribution': {'ungrounded': 129, 'HGNC:9967': 147},
 'f1': {'mean': 0.96687, 'std': 0.010014},
 'precision': {'mean': 0.94879, 'std': 0.025617},
 'recall': {'mean': 0.986437, 'std': 0.016615},
 'HGNC:9967': {'f1': {'mean': 0.96687, 'std': 0.010014},
  'pr': {'mean': 0.986437, 'std': 0.016615},
  'rc': {'mean': 0.94879, 'std': 0.025617}},
 'ungrounded': {'f1': {'mean': 0.959982, 'std': 0.013394},
  'pr': {'mean': 0.937846, 'std': 0.0311},
  'rc': {'mean': 0.984259, 'std': 0.019334}}}

In [23]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [24]:
disamb.dump(model_name, results_path)

In [25]:
print(disamb.info())

Disambiguation model for RET

Produces the disambiguations:
	RET*	HGNC:9967

Class level metrics:
--------------------
Grounding	Count	F1     
RET*	147	0.96687
Ungrounded	129	0.95998

Weighted Metrics:
-----------------
	F1 score:	0.96687
	Precision:	0.94879
	Recall:		0.98644

* Positive labels
See Docstring for explanation



In [26]:
model_to_s3(disamb)